In [3]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gc
import time
import datetime

In [4]:
# データ読み込み、各月の結合、欠損値を含む行の削除
# 1 : neg_count, neu_count, pos_count + end_price, trend
def read_data1(timespan,n):
    print("[read開始]\n")
    
    files = [f'tweet-svm/{timespan}/2021-01.csv',
             f'tweet-svm/{timespan}/2021-02.csv',
             f'tweet-svm/{timespan}/2021-03.csv',
             f'tweet-svm/{timespan}/2021-04.csv',
             f'tweet-svm/{timespan}/2021-05.csv',
             f'tweet-svm/{timespan}/2021-06.csv',
             f'tweet-svm/{timespan}/2021-07.csv']
    datas = []
    df = pd.DataFrame()
    for file in files:
        data = pd.read_csv(file, usecols = ['section','neg_count','neu_count','pos_count','open_price','trend'])
        datas.append(data)

    # ファイルの結合
    df = pd.concat(datas).reset_index(drop=True)
    df = df.drop(columns=['section'])
    
    # カラム名変更(区間No.割り当て)
    df = df.rename(columns={"neg_count":"neg_count(n)", 
                        "neu_count":"neu_count(n)", 
                        "pos_count":"pos_count(n)", 
                        "open_price":"open_price(n)",
                        "trend":"trend(n)"})
    
    print('欠損値削除前の総データ数：{}'.format(len(df)))
    
    # 目的変数
    df['trend(n+1)'] = df['trend(n)'].shift(-1)

    # 説明変数
    df['end_price(n)'] = df['open_price(n)'].shift(-1)
    if n >= 2:
        for i in range(1,n):
            df[f'neg_count(n-{i})'] = df['neg_count(n)'].shift(i)
            df[f'neu_count(n-{i})'] = df['neu_count(n)'].shift(i)
            df[f'pos_count(n-{i})'] = df['pos_count(n)'].shift(i)
            df[f'trend(n-{i})'] = df['trend(n)'].shift(i)
            df[f'end_price(n-{i})'] = df['end_price(n)'].shift(i)

    df = df.drop(columns=['open_price(n)'])

    # 欠損値を含む行を削除
    df = df.dropna(how='any')
    print('欠損値削除後の総データ数：{}\n'.format(len(df)))
    
    print("[read終了]")
    
    return df

In [5]:
# データ読み込み、各月の結合、欠損値を含む行の削除
# 2 : com_ave, tweet_count　+ end_price, trend
def read_data2(timespan,n):
    print("[read開始]\n")
    
    files = [f'tweet-svm/{timespan}/2021-01.csv',
             f'tweet-svm/{timespan}/2021-02.csv',
             f'tweet-svm/{timespan}/2021-03.csv',
             f'tweet-svm/{timespan}/2021-04.csv',
             f'tweet-svm/{timespan}/2021-05.csv',
             f'tweet-svm/{timespan}/2021-06.csv',
             f'tweet-svm/{timespan}/2021-07.csv']
    datas = []
    df = pd.DataFrame()
    for file in files:
        data = pd.read_csv(file, usecols = ['section','com_ave','tweet_count','open_price','trend'])
        datas.append(data)

    # ファイルの結合
    df = pd.concat(datas).reset_index(drop=True)
    df = df.drop(columns=['section'])
    
    # カラム名変更(区間No.割り当て)
    df = df.rename(columns={"com_ave":"com_ave(n)", 
                            "tweet_count":"tweet_count(n)", 
                            "open_price":"open_price(n)",
                            "trend":"trend(n)"})
    
    print('欠損値削除前の総データ数：{}'.format(len(df)))
    
    # 目的変数
    df['trend(n+1)'] = df['trend(n)'].shift(-1)

    # 説明変数
    df['end_price(n)'] = df['open_price(n)'].shift(-1)
    if n >= 2:
        for i in range(1,n):
            df[f'com_ave(n-{i})'] = df['com_ave(n)'].shift(i)
            df[f'tweet_count(n-{i})'] = df['tweet_count(n)'].shift(i)
            df[f'trend(n-{i})'] = df['trend(n)'].shift(i)
            df[f'end_price(n-{i})'] = df['end_price(n)'].shift(i)

    df = df.drop(columns=['open_price(n)'])

    # 欠損値を含む行を削除
    df = df.dropna(how='any')
    print('欠損値削除後の総データ数：{}\n'.format(len(df)))
    
    print("[read終了]")
    
    return df

In [6]:
# データ読み込み、各月の結合、欠損値を含む行の削除
# 3 : neg_count, neu_count, pos_count + com_ave, tweet_count + end_price, trend
def read_data3(timespan,n):
    print("[read開始]\n")
    
    files = [f'tweet-svm/{timespan}/2021-01.csv',
             f'tweet-svm/{timespan}/2021-02.csv',
             f'tweet-svm/{timespan}/2021-03.csv',
             f'tweet-svm/{timespan}/2021-04.csv',
             f'tweet-svm/{timespan}/2021-05.csv',
             f'tweet-svm/{timespan}/2021-06.csv',
             f'tweet-svm/{timespan}/2021-07.csv']
    datas = []
    df = pd.DataFrame()
    for file in files:
        data = pd.read_csv(file)
        datas.append(data)

    # ファイルの結合
    df = pd.concat(datas).reset_index(drop=True)
    df = df.drop(columns=['section'])
    
    # カラム名変更(区間No.割り当て)
    df = df.rename(columns={"neg_count":"neg_count(n)", 
                            "neu_count":"neu_count(n)", 
                            "pos_count":"pos_count(n)",
                            "com_ave":"com_ave(n)", 
                            "tweet_count":"tweet_count(n)", 
                            "open_price":"open_price(n)",
                            "trend":"trend(n)"})
    
    print('欠損値削除前の総データ数：{}'.format(len(df)))
    
    # 目的変数
    df['trend(n+1)'] = df['trend(n)'].shift(-1)

    # 説明変数
    df['end_price(n)'] = df['open_price(n)'].shift(-1)
    if n >= 2:
        for i in range(1,n):
            df[f'neg_count(n-{i})'] = df['neg_count(n)'].shift(i)
            df[f'neu_count(n-{i})'] = df['neu_count(n)'].shift(i)
            df[f'pos_count(n-{i})'] = df['pos_count(n)'].shift(i)
            df[f'com_ave(n-{i})'] = df['com_ave(n)'].shift(i)
            df[f'tweet_count(n-{i})'] = df['tweet_count(n)'].shift(i)
            df[f'trend(n-{i})'] = df['trend(n)'].shift(i)
            df[f'end_price(n-{i})'] = df['end_price(n)'].shift(i)
    
    df = df.drop(columns=['open_price(n)'])

    # 欠損値を含む行を削除
    df = df.dropna(how='any')
    print('欠損値削除後の総データ数：{}\n'.format(len(df)))
    
    print("[read終了]")
    
    return df

In [7]:
# データ読み込み、各月の結合、欠損値を含む行の削除
# 4 : neg_count, neu_count, pos_count
def read_data4(timespan,n):
    print("[read開始]\n")
    
    files = [f'tweet-svm/{timespan}/2021-01.csv',
             f'tweet-svm/{timespan}/2021-02.csv',
             f'tweet-svm/{timespan}/2021-03.csv',
             f'tweet-svm/{timespan}/2021-04.csv',
             f'tweet-svm/{timespan}/2021-05.csv',
             f'tweet-svm/{timespan}/2021-06.csv',
             f'tweet-svm/{timespan}/2021-07.csv']
    datas = []
    df = pd.DataFrame()
    for file in files:
        data = pd.read_csv(file, usecols = ['section','neg_count','neu_count','pos_count','trend'])
        datas.append(data)

    # ファイルの結合
    df = pd.concat(datas).reset_index(drop=True)
    df = df.drop(columns=['section'])
    
    # カラム名変更(区間No.割り当て)
    df = df.rename(columns={"neg_count":"neg_count(n)", 
                            "neu_count":"neu_count(n)", 
                            "pos_count":"pos_count(n)",
                            "trend":"trend(n)"})
    
    print('欠損値削除前の総データ数：{}'.format(len(df)))
    
    # 目的変数
    df['trend(n+1)'] = df['trend(n)'].shift(-1)

    # 説明変数
    if n >= 2:
        for i in range(1,n):
            df[f'neg_count(n-{i})'] = df['neg_count(n)'].shift(i)
            df[f'neu_count(n-{i})'] = df['neu_count(n)'].shift(i)
            df[f'pos_count(n-{i})'] = df['pos_count(n)'].shift(i)
    
    df = df.drop(columns=['trend(n)'])

    # 欠損値を含む行を削除
    df = df.dropna(how='any')
    print('欠損値削除後の総データ数：{}\n'.format(len(df)))
    
    print("[read終了]")
    
    return df

In [8]:
# データ読み込み、各月の結合、欠損値を含む行の削除
# 5 : com_ave, tweet_count
def read_data5(timespan,n):
    print("[read開始]\n")
    
    files = [f'tweet-svm/{timespan}/2021-01.csv',
             f'tweet-svm/{timespan}/2021-02.csv',
             f'tweet-svm/{timespan}/2021-03.csv',
             f'tweet-svm/{timespan}/2021-04.csv',
             f'tweet-svm/{timespan}/2021-05.csv',
             f'tweet-svm/{timespan}/2021-06.csv',
             f'tweet-svm/{timespan}/2021-07.csv']
    datas = []
    df = pd.DataFrame()
    for file in files:
        data = pd.read_csv(file, usecols = ['section','com_ave','tweet_count','trend'])
        datas.append(data)

    # ファイルの結合
    df = pd.concat(datas).reset_index(drop=True)
    df = df.drop(columns=['section'])
    
    # カラム名変更(区間No.割り当て)
    df = df.rename(columns={"com_ave":"com_ave(n)", 
                            "tweet_count":"tweet_count(n)", 
                            "trend":"trend(n)"})
    
    print('欠損値削除前の総データ数：{}'.format(len(df)))
    
    # 目的変数
    df['trend(n+1)'] = df['trend(n)'].shift(-1)

    # 説明変数
    if n >= 2:
        for i in range(1,n):
            df[f'com_ave(n-{i})'] = df['com_ave(n)'].shift(i)
            df[f'tweet_count(n-{i})'] = df['tweet_count(n)'].shift(i)
    
    df = df.drop(columns=['trend(n)'])

    # 欠損値を含む行を削除
    df = df.dropna(how='any')
    print('欠損値削除後の総データ数：{}\n'.format(len(df)))
    
    print("[read終了]")
    
    return df

In [9]:
# データ読み込み、各月の結合、欠損値を含む行の削除
# 6 : neg_count, neu_count, pos_count + com_ave, tweet_count
def read_data6(timespan,n):
    print("[read開始]\n")
    
    files = [f'tweet-svm/{timespan}/2021-01.csv',
             f'tweet-svm/{timespan}/2021-02.csv',
             f'tweet-svm/{timespan}/2021-03.csv',
             f'tweet-svm/{timespan}/2021-04.csv',
             f'tweet-svm/{timespan}/2021-05.csv',
             f'tweet-svm/{timespan}/2021-06.csv',
             f'tweet-svm/{timespan}/2021-07.csv']
    datas = []
    df = pd.DataFrame()
    for file in files:
        data = pd.read_csv(file, usecols = ['section','neg_count','neu_count','pos_count','com_ave','tweet_count','trend'])
        datas.append(data)

    # ファイルの結合
    df = pd.concat(datas).reset_index(drop=True)
    df = df.drop(columns=['section'])
    
    # カラム名変更(区間No.割り当て)
    df = df.rename(columns={"neg_count":"neg_count(n)", 
                            "neu_count":"neu_count(n)", 
                            "pos_count":"pos_count(n)",
                            "com_ave":"com_ave(n)", 
                            "tweet_count":"tweet_count(n)", 
                            "open_price":"open_price(n)",
                            "trend":"trend(n)"})
    
    print('欠損値削除前の総データ数：{}'.format(len(df)))
    
    # 目的変数
    df['trend(n+1)'] = df['trend(n)'].shift(-1)

    # 説明変数
    if n >= 2:
        for i in range(1,n):
            df[f'neg_count(n-{i})'] = df['neg_count(n)'].shift(i)
            df[f'neu_count(n-{i})'] = df['neu_count(n)'].shift(i)
            df[f'pos_count(n-{i})'] = df['pos_count(n)'].shift(i)
            df[f'com_ave(n-{i})'] = df['com_ave(n)'].shift(i)
            df[f'tweet_count(n-{i})'] = df['tweet_count(n)'].shift(i)
    
    df = df.drop(columns=['trend(n)'])

    # 欠損値を含む行を削除
    df = df.dropna(how='any')
    print('欠損値削除後の総データ数：{}\n'.format(len(df)))
    
    print("[read終了]")
    
    return df

In [10]:
# データ読み込み、各月の結合、欠損値を含む行の削除
# 7 : end_price, trend
def read_data7(timespan,n):
    print("[read開始]\n")
    
    files = [f'tweet-svm/{timespan}/2021-01.csv',
             f'tweet-svm/{timespan}/2021-02.csv',
             f'tweet-svm/{timespan}/2021-03.csv',
             f'tweet-svm/{timespan}/2021-04.csv',
             f'tweet-svm/{timespan}/2021-05.csv',
             f'tweet-svm/{timespan}/2021-06.csv',
             f'tweet-svm/{timespan}/2021-07.csv']
    datas = []
    df = pd.DataFrame()
    for file in files:
        data = pd.read_csv(file, usecols = ['section','open_price','trend'])
        datas.append(data)

    # ファイルの結合
    df = pd.concat(datas).reset_index(drop=True)
    df = df.drop(columns=['section'])
    
    # カラム名変更(区間No.割り当て)
    df = df.rename(columns={"open_price":"open_price(n)",
                            "trend":"trend(n)"})
    
    print('欠損値削除前の総データ数：{}'.format(len(df)))
    
    # 目的変数
    df['trend(n+1)'] = df['trend(n)'].shift(-1)

    # 説明変数
    df['end_price(n)'] = df['open_price(n)'].shift(-1)
    if n >= 2:
        for i in range(1,n):
            df[f'trend(n-{i})'] = df['trend(n)'].shift(i)
            df[f'end_price(n-{i})'] = df['end_price(n)'].shift(i)
    
    df = df.drop(columns=['open_price(n)'])

    # 欠損値を含む行を削除
    df = df.dropna(how='any')
    print('欠損値削除後の総データ数：{}\n'.format(len(df)))
    
    print("[read終了]")
    
    return df

In [11]:
# スケーリング x' = (x - xmin)/(xmax - xmin)
def scaling(x_train,x_test):
    print("[scaling開始]")
    
    scaler = MinMaxScaler().fit(x_train)

    x_train_scaled = scaler.transform(x_train)
    x_test_scaled  = scaler.transform(x_test)
    
    print("[scalig終了]")
    
    return x_train_scaled, x_test_scaled

In [12]:
# 学習 One-versus-the-rest
# Default ver. C=1, gamma=auto
def default_svm(x_train, y_train, x_test):
    print("[学習開始(Default)]")
    
    model = OneVsRestClassifier(SVC())
    model.fit(x_train, y_train)
    pred_y = model.predict(x_test)
    
    print("[学習終了(Default)]\n")
    
    return pred_y

In [ ]:
# C,gammmaの最適値探索ver.
'''
def best_svm(x_train, y_train, x_test):
    model = OneVsRestClassifier(SVC())
    C_params = np.logspace(-4, 4, 5) #10^(-4) ~ 10^4 までで均等に5つの値
    gamma_params = np.logspace(-4, 4, 5)

    parameters = {'estimator__C': C_params,
                  'estimator__gamma': gamma_params}

    model_tuning = GridSearchCV(estimator = model,
                                param_grid = parameters,
                                n_jobs = -1,
                                verbose = 3
    )
    model_tuning.fit(x_train, y_train)
    pred_y2 = model_tuning.predict(test_x)

    # Best parameter
    model_tuning.best_params_
    
    return pred_y2
    
'''

In [13]:
# main
# all実行 (a=1~7, n=1~10, timespan×7の490通り)
alist = {1:'neg_count, neu_count, pos_count + end_price, trend',
         2:'com_ave, tweet_count + end_price, trend',
         3:'neg_count, neu_count, pos_count + com_ave, tweet_count + end_price, trend',
         4:'neg_count, neu_count, pos_count',
         5:'com_ave, tweet_count',
         6:'neg_count, neu_count, pos_count + com_ave, tweet_count',
         7:'end_price, trend'}
s = ('学習に用いる感情データの種類 a\n' 
     '1 : neg_count, neu_count, pos_count + end_price, trend\n'
     '2 : com_ave, tweet_count + end_price, trend\n'
     '3 : neg_count, neu_count, pos_count + com_ave, tweet_count + end_price, trend\n'
     '4 : neg_count, neu_count, pos_count\n'
     '5 : com_ave, tweet_count\n'
     '6 : neg_count, neu_count, pos_count + com_ave, tweet_count\n'
     '7 : end_price, trend')
print(s) 
print("学習に用いるデータの範囲 n\n(例1) n = 1 → 区間[n]\n(例2) n = 2 → 区間[n-1, n]")

alist_ = [1,2,3,4,5,6,7]
nlist = [1,2,3,4,5,6,7,8,9,10]
tlist = ['1d','12h','4h','1h','30m','15m','5m']
aculist = {}

for a in alist_:
    for n in nlist:
        for timespan in tlist:
            print("********************{}：開始********************\n".format(timespan))
            dt_start = datetime.datetime.now()
            print(f'開始時刻：{dt_start}\n')

            print(f'(a,n) = ({a},{n})')

            result = []

            #データの読み込み
            if a == 1:
                data = read_data1(timespan,n)
            elif a == 2:
                data = read_data2(timespan,n)
            elif a == 3:
                data = read_data3(timespan,n)
            elif a == 4:
                data = read_data4(timespan,n)
            elif a == 5:
                data = read_data5(timespan,n)
            elif a == 6:
                data = read_data6(timespan,n)
            elif a == 7:
                data = read_data7(timespan,n)
                
            # 説明変数、目的変数をセット
            x = data.iloc[:, data.columns!='trend(n+1)']
            y = data.loc[:, 'trend(n+1)']

            #データを訓練用とテスト用に分割
            #7か月のうち、1/7をテストに、6/7を訓練にランダム分割
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 1/7, random_state=0)

            # スケーリング
            x_train_scaled, x_test_scaled = scaling(x_train, x_test)

            # 学習
            y_pred = default_svm(x_train_scaled, y_train, x_test_scaled)

            # 結果表示
            print ('accuracy(Default) : {:.5f}\n'.format(accuracy_score(y_test, y_pred)))
            #print ('最適地探索: {:.5f}'.format(accuracy_score(test_y, pred_y2)))

            # 混同行列
            print(confusion_matrix(y_test, y_pred))

            # precision, recall, f1-score, support
            print(classification_report(y_test, y_pred, target_names = ['neg','neu','pos']))

            dt_end = datetime.datetime.now()
            print(f'終了時刻：{dt_end}\n')

            # 結果を格納
            aculist[f'{timespan}'] = accuracy_score(y_test, y_pred)
            df_log = pd.read_csv('tweet-svm/svm_log.csv')
            s = pd.Series([dt_start,
                           dt_end,
                           timespan,
                           alist[a],
                           n,
                           len(x.columns.values),
                           len(data),
                           len(y_train),
                           len(y_test),
                           accuracy_score(y_test, y_pred),
                           confusion_matrix(y_test, y_pred),
                           classification_report(y_test, y_pred,target_names=['neg','neu','pos'], output_dict=True)],
                    index=['start_time',
                           'end_time',
                           'timespan',
                           'kind_of_data',
                           'range_of_data',
                           'attribute',
                           'all_data',
                           'training_data',
                           'test_data',
                           'accuracy',
                           'confusion_matrix',
                           'score_report'])
            df_log = df_log.append(s, ignore_index=True)
            df_log.to_csv('tweet-svm/svm_log.csv',index=False)

            # メモリ開放
            del data,x,y,x_train,x_test,y_train,y_test,x_train_scaled,x_test_scaled,y_pred,dt_start,dt_end,df_log
            gc.collect()

            print("********************{}：終了********************\n".format(timespan))
    
print(f'正答率\n{aculist}')

学習に用いる感情データの種類 a
1 : neg_count, neu_count, pos_count + end_price, trend
2 : com_ave, tweet_count + end_price, trend
3 : neg_count, neu_count, pos_count + com_ave, tweet_count + end_price, trend
4 : neg_count, neu_count, pos_count
5 : com_ave, tweet_count
6 : neg_count, neu_count, pos_count + com_ave, tweet_count
7 : end_price, trend
学習に用いるデータの範囲 n
(例1) n = 1 → 区間[n]
(例2) n = 2 → 区間[n-1, n]
********************1d：開始********************

開始時刻：2021-10-10 14:59:57.091932

(a,n) = (1,1)
[read開始]

欠損値削除前の総データ数：212
欠損値削除後の総データ数：211

[read終了]
[scaling開始]
[scalig終了]
[学習開始(Default)]
[学習終了(Default)]

accuracy(Default) : 0.41935

[[5 4 1]
 [4 5 0]
 [2 7 3]]
              precision    recall  f1-score   support

         neg       0.45      0.50      0.48        10
         neu       0.31      0.56      0.40         9
         pos       0.75      0.25      0.38        12

    accuracy                           0.42        31
   macro avg       0.51      0.44      0.42        31
weighted avg       0

In [2]:
#log リセット
#'''
df = pd.DataFrame(columns=['start_time',
                   'end_time',
                   'timespan',
                   'kind_of_data',
                   'range_of_data',
                   'attribute',
                   'all_data',
                   'training_data',
                   'test_data',
                   'accuracy',
                   'confusion_matrix',
                   'score_report'])
df.to_csv('tweet-svm/svm_log.csv',index=False)
#'''